In [5]:
import pandas as pd
import keras
from keras.layers import Conv1D
from keras.layers import Dense
from keras.optimizers import SGD
from keras.layers import Flatten
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from keras.models import Sequential
from keras.layers import Dense

In [6]:
train = pd.read_csv('../data/acc_data/training_data.csv',header = None)

In [7]:
train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,6.791859,13.960164,-0.193163,48.522705,-0.589755,-16.461533,-1.630623,0.330035,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,6.800353,13.996486,-0.190413,48.528838,-0.604140,-16.450406,-1.691989,0.298628,-0.110716,0.610561,0.312486,-0.084510,-0.361382,-0.027366
2,6.800085,14.002038,-0.174075,48.533003,-0.599635,-16.425717,-1.718877,0.251782,-0.245641,0.267517,0.466097,0.002662,-0.055242,-0.162555
3,6.800586,14.011322,-0.175465,48.535982,-0.576815,-16.399336,-1.747659,0.225503,-0.262477,0.286367,0.261464,-0.004982,-0.092371,0.013831
4,6.813845,14.013416,-0.177808,48.531692,-0.565499,-16.385513,-1.769202,0.216323,-0.137533,0.214349,0.091333,-0.131924,-0.020837,0.023308


In [15]:
X = train.loc[:,:10]
y = train.loc[:,11:13]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [19]:
model = Sequential()
model.add(Dense(12, input_dim=11, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='sigmoid'))
epochs = 10
lrate = 0.01
decay = lrate/epochs
model.compile(loss='mean_absolute_error', optimizer='rmsprop', metrics = ['mae'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 12)                144       
_________________________________________________________________
dense_13 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_14 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 27        
Total params: 347
Trainable params: 347
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
model.fit(X_train,y_train,epochs = epochs)

Epoch 1/10
10767/10767 [==============================] - 1s 88us/step - loss: 0.2695 - mae: 0.2695
Epoch 2/10
10767/10767 [==============================] - 1s 70us/step - loss: 0.2496 - mae: 0.2496
Epoch 3/10
10767/10767 [==============================] - 1s 70us/step - loss: 0.2496 - mae: 0.2496
Epoch 4/10
10767/10767 [==============================] - 1s 55us/step - loss: 0.2496 - mae: 0.2496
Epoch 5/10
10767/10767 [==============================] - 1s 71us/step - loss: 0.2496 - mae: 0.2496
Epoch 6/10
10767/10767 [==============================] - 1s 71us/step - loss: 0.2496 - mae: 0.2496
Epoch 7/10
10767/10767 [==============================] - 1s 70us/step - loss: 0.2496 - mae: 0.2496
Epoch 8/10
10767/10767 [==============================] - 1s 76us/step - loss: 0.2496 - mae: 0.2496
Epoch 9/10
10767/10767 [==============================] - 1s 76us/step - loss: 0.2496 - mae: 0.2496
Epoch 10/10
10767/10767 [==============================] - 1s 70us/step - loss: 0.2496 - mae: 0.2496

In [21]:
_, mae = model.evaluate(X_test, y_test)
print('MAE: %.2f' % (mae))

3590/3590 [==============================] - 0s 42us/step
MAE: 0.26
